# Week 2 Cab Analysis

**The Client**

XYZ is a private firm in US. Due to remarkable growth in the Cab Industry in last few years and multiple key players in the market, it is planning for an investment in Cab industry and as per their Go-to-Market(G2M) strategy they want to understand the market before taking final decision.

**Project Delivery**

You have been provided with multiple data sets that contains information on 2 cab companies. Each file (data set) provided represents different aspects of the customer profile. XYZ is interested in using your actionable insights to help them identify the right company to make their investment.

The outcome of your delivery will be a presentation to XYZ’s Executive team. This presentation will be judged based on the visuals provided, the quality of your analysis and the value of your recommendations and insights. 

**Data:**

You have been provided 4 individual data sets. Time period of data is from 31/01/2016 to 31/12/2018.

Below are the list of datasets which are provided for the analysis

* `Cab_Data.csv` - this file includes details of transaction for 2 cab companies
* `Customer_ID.csv` - this is a mapping table that contains a unique identifier which links the customer's demographic details
* `Transaction_ID.csv` - this is a mapping table that contains transaction to customer mappings and payment mode
* `City.csv` - this file contains list of US cities, their population and number of cab users.

* Review the Source Documentation
* Understand the field names and data types
* Identify relationships across the files
* Field/feature transformations
* Determine which files should be joined versus which ones should be appended
* Create master data and explain the relationship
* Identify and remove duplicates
* Perform other analysis like NA value and outlier detection

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
cab_data_path = 'data/Cab_Data.csv'
city_data_path = 'data/City.csv'
customer_id_path = 'data/Customer_ID.csv'
transaction_id_path = 'data/Transaction_ID.csv'

In [3]:
cab_df = pd.read_csv(cab_data_path)

print(cab_df.shape)
cab_df.head()

(359392, 7)


,Transaction ID,Date of Travel,Company,City,KM Travelled,Price Charged,Cost of Trip
0,10000011,42377,Pink Cab,ATLANTA GA,30.45,370.95,313.635
1,10000012,42375,Pink Cab,ATLANTA GA,28.62,358.52,334.854
2,10000013,42371,Pink Cab,ATLANTA GA,9.04,125.20,97.632
3,10000014,42376,Pink Cab,ATLANTA GA,33.17,377.40,351.602
4,10000015,42372,Pink Cab,ATLANTA GA,8.73,114.62,97.776


In [4]:
city_df = pd.read_csv(city_data_path)

print(city_df.shape)
city_df.head()

(20, 3)


,City,Population,Users
0,NEW YORK NY,"8,405,837","302,149"
1,CHICAGO IL,"1,955,130","164,468"
2,LOS ANGELES CA,"1,595,037","144,132"
3,MIAMI FL,"1,339,155","17,675"
4,SILICON VALLEY,"1,177,609","27,247"


In [5]:
customer_df = pd.read_csv(customer_id_path)

print(customer_df.shape)
customer_df.head()

(49171, 4)


,Customer ID,Gender,Age,Income (USD/Month)
0,29290,Male,28,10813
1,27703,Male,27,9237
2,28712,Male,53,11242
3,28020,Male,23,23327
4,27182,Male,33,8536


In [6]:
transaction_df = pd.read_csv(transaction_id_path)

print(transaction_df.shape)
transaction_df.head()

(440098, 3)


,Transaction ID,Customer ID,Payment_Mode
0,10000011,29290,Card
1,10000012,27703,Card
2,10000013,28712,Cash
3,10000014,28020,Cash
4,10000015,27182,Card


## Field Names and Data Types

### `cab_df`

In [7]:
print(cab_df.shape)
cab_df.head()

(359392, 7)


,Transaction ID,Date of Travel,Company,City,KM Travelled,Price Charged,Cost of Trip
0,10000011,42377,Pink Cab,ATLANTA GA,30.45,370.95,313.635
1,10000012,42375,Pink Cab,ATLANTA GA,28.62,358.52,334.854
2,10000013,42371,Pink Cab,ATLANTA GA,9.04,125.20,97.632
3,10000014,42376,Pink Cab,ATLANTA GA,33.17,377.40,351.602
4,10000015,42372,Pink Cab,ATLANTA GA,8.73,114.62,97.776


In [8]:
cab_df.dtypes

Transaction ID      int64
Date of Travel      int64
Company            object
City               object
KM Travelled      float64
Price Charged     float64
Cost of Trip      float64
dtype: object

* `Company` and `City` are object type; 
* It would be more convenient to convert them to categorical/string type.

In [9]:
cab_df['Company'] = pd.Categorical(cab_df['Company'])
cab_df['City'] = pd.Categorical(cab_df['City'])

In [10]:
cab_df.dtypes

Transaction ID       int64
Date of Travel       int64
Company           category
City              category
KM Travelled       float64
Price Charged      float64
Cost of Trip       float64
dtype: object

Looks like `Date of Travel` column is represented in specific numeric format, probably Excel's date serial number. Let's convert it into a standard date format:

In [26]:
cab_df['Date of Travel'] = pd.to_datetime(cab_df['Date of Travel'], origin = '1899-12-30', unit = 'D')
cab_df['Date of Travel']

0        2016-01-08
1        2016-01-06
2        2016-01-02
3        2016-01-07
4        2016-01-03
            ...    
359387   2018-01-08
359388   2018-01-04
359389   2018-01-05
359390   2018-01-05
359391   2018-01-02
Name: Date of Travel, Length: 359392, dtype: datetime64[ns]

In [34]:
cab_df['Date of Travel'].describe()

count                           359392
mean     2017-08-17 01:37:55.042293760
min                2016-01-02 00:00:00
25%                2016-11-23 00:00:00
50%                2017-09-10 00:00:00
75%                2018-05-12 00:00:00
max                2018-12-31 00:00:00
Name: Date of Travel, dtype: object

The data covers the period from 2nd January 2016 to 31st December 2018.

### `city_df`

In [11]:
print(city_df.shape)
city_df.head()

(20, 3)


,City,Population,Users
0,NEW YORK NY,"8,405,837","302,149"
1,CHICAGO IL,"1,955,130","164,468"
2,LOS ANGELES CA,"1,595,037","144,132"
3,MIAMI FL,"1,339,155","17,675"
4,SILICON VALLEY,"1,177,609","27,247"


In [12]:
city_df.dtypes

City          object
Population    object
Users         object
dtype: object

* Let's convert `City` to string; 
* `Population` and `Users` will be converted to numeric

In [13]:
# Convert 'City' to string
city_df['City'] = city_df['City'].astype(str)

# Convert 'Population' and 'Users' to numeric
city_df['Population'] = pd.to_numeric(city_df['Population'].str.replace(',', ''))
city_df['Users'] = pd.to_numeric(city_df['Users'].str.replace(',', ''))

city_df.dtypes

City          object
Population     int64
Users          int64
dtype: object

### `customer_df`

In [14]:
print(customer_df.shape)
customer_df.head()

(49171, 4)


,Customer ID,Gender,Age,Income (USD/Month)
0,29290,Male,28,10813
1,27703,Male,27,9237
2,28712,Male,53,11242
3,28020,Male,23,23327
4,27182,Male,33,8536


In [15]:
customer_df.dtypes

Customer ID            int64
Gender                object
Age                    int64
Income (USD/Month)     int64
dtype: object

`Gender` column should be converted to categorical

In [19]:
customer_df['Gender'] = pd.Categorical(customer_df['Gender'])

customer_df.dtypes

Customer ID              int64
Gender                category
Age                      int64
Income (USD/Month)       int64
dtype: object

### `transaction_df`

In [16]:
print(transaction_df.shape)
transaction_df.head()

(440098, 3)


,Transaction ID,Customer ID,Payment_Mode
0,10000011,29290,Card
1,10000012,27703,Card
2,10000013,28712,Cash
3,10000014,28020,Cash
4,10000015,27182,Card


In [17]:
transaction_df.dtypes

Transaction ID     int64
Customer ID        int64
Payment_Mode      object
dtype: object

* `Payment_Mode` will be converted to categorical

In [21]:
transaction_df['Payment_Mode'] = pd.Categorical(transaction_df['Payment_Mode'])

transaction_df.dtypes

Transaction ID       int64
Customer ID          int64
Payment_Mode      category
dtype: object

In [12]:
cab_df['Transaction ID'].is_unique

True

In [16]:
cab_df['City'].value_counts()

City
NEW YORK NY       99885
CHICAGO IL        56625
LOS ANGELES CA    48033
WASHINGTON DC     43737
BOSTON MA         29692
SAN DIEGO CA      20488
SILICON VALLEY     8519
SEATTLE WA         7997
ATLANTA GA         7557
DALLAS TX          7017
MIAMI FL           6454
AUSTIN TX          4896
ORANGE COUNTY      3982
DENVER CO          3825
NASHVILLE TN       3010
SACRAMENTO CA      2367
PHOENIX AZ         2064
TUCSON AZ          1931
PITTSBURGH PA      1313
Name: count, dtype: int64

## Relationships between variables:
* `Customer_ID` is the same in both `customer_df` and `transaction_df`
* `Transaction_ID` is the same in both `cab_df` and `transaction_df`